In [1]:
from lite_llm import LiteLLMService, LiteLLMEmbeddingInput, LiteLLMSetting 
from pydantic import HttpUrl, SecretStr

In [2]:
litellm_service=LiteLLMService(
    litellm_setting=LiteLLMSetting(
        url=HttpUrl("http://localhost:9510"),
        token=SecretStr("abc123"),
        model="gemini-2.5-flash",
        frequency_penalty=0.0,
        n=1,
        temperature=0.0,
        top_p=1.0,
        max_completion_tokens=10000,
        dimension=1536,
        embedding_model="gemini-embedding"
    )
)
litellm_service

LiteLLMService(litellm_setting=LiteLLMSetting(url=HttpUrl('http://localhost:9510/'), token=SecretStr('**********'), model='gemini-2.5-flash', frequency_penalty=0.0, n=1, temperature=0.0, top_p=1.0, max_completion_tokens=10000, dimension=1536, embedding_model='gemini-embedding'), async_client=None, client=None)

In [3]:
output = await litellm_service.embedding_llm_async(
    inputs=LiteLLMEmbeddingInput(
        text="Hello, world!"
    )
)

In [4]:
len(output.embedding)

1536

In [18]:
options = [
    "By explicitly computing feature vectors in a higher-dimensional space and then applying a linear SVM.",
    "By replacing the inner product of mapped feature vectors with a kernel function, avoiding explicit computation in high-dimensional space.",  # correct
    "By directly adjusting the weights `w` and bias `b` to create a non-linear decision boundary in the input space.",
    "By reducing the dimensionality of the input data before applying a linear SVM."
]

import numpy as np

# Get embeddings for each option
option_embeddings = []
for option in options:
    embedding_output = await litellm_service.embedding_llm_async(
        inputs=LiteLLMEmbeddingInput(text=option)
    )
    option_embeddings.append(np.array(embedding_output.embedding))

print(f"Generated {len(option_embeddings)} embeddings, each with dimension {len(option_embeddings[0])}")

Generated 4 embeddings, each with dimension 1536


In [19]:
import numpy as np
from typing import List

def cosine_similarity(vector1: np.ndarray, vector2: np.ndarray) -> float:
    """
    Calculate cosine similarity between two vectors.
    
    Args:
        vector1: First vector as numpy array
        vector2: Second vector as numpy array
    
    Returns:
        Cosine similarity value between -1 and 1
    """
    # Calculate dot product
    dot_product = np.dot(vector1, vector2)
    
    # Calculate magnitudes (L2 norms)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    # Avoid division by zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    # Calculate cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    
    return similarity

def compute_similarity_matrix(texts: List[str], embeddings: List[np.ndarray]) -> np.ndarray:
    """
    Compute a similarity matrix for a list of text embeddings.
    
    Args:
        texts: List of text strings
        embeddings: List of embedding vectors
    
    Returns:
        NxN similarity matrix where N is the number of texts
    """
    n = len(embeddings)
    similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])
    
    return similarity_matrix

In [20]:
# Compute the 4x4 similarity matrix
similarity_matrix = compute_similarity_matrix(options, option_embeddings)

print("4x4 Cosine Similarity Matrix:")
print("=" * 50)

# Print matrix with proper formatting
for i in range(len(options)):
    row_str = ""
    for j in range(len(options)):
        row_str += f"{similarity_matrix[i, j]:.4f}  "
    print(f"Option {i+1}: {row_str}")

print("\nDetailed Matrix with Labels:")
print("=" * 80)

# Print with option labels for clarity
for i, option1 in enumerate(options):
    print(f"\nOption {i+1}: {option1[:50]}...")
    for j, option2 in enumerate(options):
        print(f"  vs Option {j+1}: {similarity_matrix[i, j]:.4f}")

# Also display as a clean numpy array
print(f"\nNumpy Array:\n{similarity_matrix}")
print(f"\nMatrix Shape: {similarity_matrix.shape}")

4x4 Cosine Similarity Matrix:
Option 1: 1.0000  0.8497  0.6905  0.7976  
Option 2: 0.8497  1.0000  0.6641  0.7329  
Option 3: 0.6905  0.6641  1.0000  0.6729  
Option 4: 0.7976  0.7329  0.6729  1.0000  

Detailed Matrix with Labels:

Option 1: By explicitly computing feature vectors in a highe...
  vs Option 1: 1.0000
  vs Option 2: 0.8497
  vs Option 3: 0.6905
  vs Option 4: 0.7976

Option 2: By replacing the inner product of mapped feature v...
  vs Option 1: 0.8497
  vs Option 2: 1.0000
  vs Option 3: 0.6641
  vs Option 4: 0.7329

Option 3: By directly adjusting the weights `w` and bias `b`...
  vs Option 1: 0.6905
  vs Option 2: 0.6641
  vs Option 3: 1.0000
  vs Option 4: 0.6729

Option 4: By reducing the dimensionality of the input data b...
  vs Option 1: 0.7976
  vs Option 2: 0.7329
  vs Option 3: 0.6729
  vs Option 4: 1.0000

Numpy Array:
[[1.         0.84968432 0.69050761 0.79757521]
 [0.84968432 1.         0.66406493 0.73291624]
 [0.69050761 0.66406493 1.         0.67288202]
 

In [1]:
import chromadb
from lite_llm import LiteLLMEmbeddingInput, LiteLLMService, LiteLLMSetting
from pydantic import HttpUrl, SecretStr

litellm_setting=LiteLLMSetting(
    url=HttpUrl("http://localhost:9510"),
    token=SecretStr("abc123"),
    model="gemini-2.5-flash",
    frequency_penalty=0.0,
    n=1,
    temperature=0.0,
    top_p=1.0,
    max_completion_tokens=10000,
    dimension=1536,
    embedding_model="gemini-embedding"
)

litellm_service = LiteLLMService(litellm_setting=litellm_setting)
client = chromadb.PersistentClient(path="./vector_database/rl2025")
collection = client.get_or_create_collection(name="questions")

In [3]:
from pathlib import Path

file_path = Path("rl2025/mcqs.json")
absolute_path = file_path.resolve()
print(absolute_path)


/home/lehoangvu/KLTN/rl2025/mcqs.json


In [7]:
import json 
from uuid import uuid4
from tqdm import tqdm
import time 


with open("/home/lehoangvu/KLTN/services/generation/src/generation/shared/static_files/rl2025/mcqs.json") as f:
    data = json.load(f)

for item in tqdm(data):
    embedding = await litellm_service.embedding_llm_async(
        inputs=LiteLLMEmbeddingInput(
            text=item['question']
        )
    )
    options = [
        f"{key}. {value}"
        for key, value in item['options'].items()
    ]
    collection.add(
        ids=[str(uuid4())],
        embeddings=[embedding.embedding],
        documents=[item['question']],   # text để search
        metadatas=[{
            "options": "\n".join(options),
            "answer": item['answer'],
            "explanation": item['explanation'],
        }]
    )
    time.sleep(1)

100%|██████████| 54/54 [01:13<00:00,  1.36s/it]


In [2]:
query = "Agent trong reinforcement learning là gì?"

embeddings = await litellm_service.embedding_llm_async(
    inputs=LiteLLMEmbeddingInput(
        text=query
    )
)
results = collection.query(
    query_embeddings=[embeddings.embedding],
    n_results=3,
)

results

{'ids': [['17c7414e-5923-4042-9319-d0cf48f2ad6f',
   'a55249c6-1670-4abb-acba-5649f88c7a1c',
   '8b244a37-c701-4cfa-834a-6df2e32cbda2']],
 'embeddings': None,
 'documents': [['Agent trong học tăng cường là gì?',
   'Thành phần nào trong học tăng cường xác định hành vi của agent?',
   'Bạn biểu diễn trạng thái của agent trong học tăng cường bằng gì?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'explanation': 'Agent là một thực thể thực hiện hành động và khám phá môi trường.',
    'answer': 'Agent là một thực thể khám phá môi trường.',
    'options': 'A. Agent là tình huống mà trong đó phần thưởng được trao đổi\nB. Agent là một giá trị đơn giản trong học tăng cường.\nC. Agent là một thực thể khám phá môi trường.'},
   {'options': 'A. Chính sách\nB. Tín hiệu phần thưởng\nC. Hàm giá trị\nD. Mô hình môi trường',
    'answer': 'Chính sách',
    'explanation': 'Chính sách (Policy) xác định hành vi của agent.'},
   {'explanation': 'Tr

In [4]:
results['documents'][0]

['Hàm merge trong MergeSort thực hiện hành động chính gì?',
 'Tại sao QuickSort thường được ưa thích hơn MergeSort khi sắp xếp mảng trong thực tế?',
 'Nguyên tắc mà các thuật toán chia để trị (divide and conquer) sử dụng là gì?']

In [5]:
results['metadatas'][0]

[{'answer': 'Gộp các mảng con đã sắp xếp',
  'explanation': 'Hàm merge kết hợp hai mảng con đã sắp xếp thành một mảng duy nhất đã được sắp xếp, đây là bước then chốt để đạt được thứ tự tổng thể.',
  'options': 'A. Chia mảng\nB. Sắp xếp các mảng con\nC. Gộp các mảng con đã sắp xếp\nD. So sánh từng phần tử'},
 {'explanation': 'QuickSort thường được ưa chuộng vì khả năng sắp xếp tại chỗ (in-place), cung cấp độ phức tạp không gian thấp hơn so với MergeSort, vốn cần bộ nhớ phụ để hợp nhất.',
  'options': 'A. Độ phức tạp không gian thấp hơn\nB. Thời gian trung bình nhanh hơn\nC. Dễ cài đặt hơn\nD. Đảm bảo sắp xếp ổn định',
  'answer': 'Độ phức tạp không gian thấp hơn'},
 {'explanation': 'Các thuật toán như MergeSort và QuickSort chia dữ liệu thành các phần nhỏ, sắp xếp độc lập các phần đó và sau đó kết hợp chúng để tạo thành kết quả đã sắp xếp.',
  'options': 'A. Chia dữ liệu thành các phần nhỏ hơn và sắp xếp từng phần độc lập\nB. Chọn phần tử ngẫu nhiên\nC. Duyệt tuyến tính\nD. Đổi chỗ trực